# Gestión de tráfico en sistemas de conducción autónoma. Un enfoque mediante lógica difusa.

Autor: Miguel Bande Rodríguez

# Importamos las librerías necesarias

In [ ]:
#!pip install -U scikit-fuzzy
#!pip install -U matplotlib imageio


In [ ]:
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import matplotlib.pyplot as plt
import pandas as pd
import imageio
import io

# Definimos las variables difusas y sus funciones de membresía

## Velocidad relativa

| Categoría       | ¿Qué significa?                                           | 
|----------------|------------------------------------------------------------|
| **negativa**   | El ADS va más rápido que el vehículo tripulado             | 
| **cero**       | Los dos vehículos van a la misma velocidad aproximadamente.| 
| **positiva**   | El vehículo tripulado se acerca más rápido                 | 


In [ ]:
# Velocidad relativa: diferencia entre la velocidad del vehículo tripulado y el vehículo autónomo (m/s)
# Rango ajustado de -20 a 20 m/s (equivale a 0 a 144 km/h, que es un límite razonable en autopistas)
velocidad_relativa = ctrl.Antecedent(np.arange(-20, 21, 1), 'velocidad_relativa')


# Definición de las funciones de membresía para la variable difusa "velocidad_relativa"
# Rango trabajado: -20 m/s a 20 m/s (velocidad del vehículo de la vía principal menos la del vehículo autónomo)

# La etiqueta 'rapido' representa situaciones en las que el vehículo autónomo va más rápido que el de la vía principal.
# Por ejemplo, si velocidad_relativa = -10 m/s, significa que el vehículo autónomo está alcanzando al otro rápidamente.
# Este conjunto trapezoidal cubre desde -20 (muy negativa) hasta 0 (igual velocidad).
velocidad_relativa['rapido'] = fuzz.trapmf(velocidad_relativa.universe, [-20, -20, -15, 0])

# La etiqueta 'igual' representa situaciones donde ambos vehículos van a velocidades similares.
# Aquí la diferencia de velocidades es cercana a cero (entre -2 y +2 m/s).
# Es útil para reconocer condiciones estables donde no se requiere ajuste agresivo.
velocidad_relativa['igual'] = fuzz.trimf(velocidad_relativa.universe, [-2, 0, 2])

# La etiqueta 'lento' representa que el vehículo de la vía principal va más rápido que el autónomo.
# Es decir, se está acercando desde atrás, lo que puede representar un riesgo si la distancia es corta.
# Este conjunto va desde 0 hasta 20 m/s, capturando diferentes grados de "acercamiento" del otro vehículo.
velocidad_relativa['lento'] = fuzz.trapmf(velocidad_relativa.universe, [0, 15, 20, 20])


# Graficamos
velocidad_relativa.view()

## Distancia de seguridad

In [ ]:
# Definición de la variable difusa "distancia_seguridad"
# Representa la separación entre el vehículo autónomo (ADS)
# y el vehículo más cercano delante (generalmente un vehículo convencional).
# Se trabaja en metros, incluyendo valores negativos cuando el ADS ya ha rebasado al otro vehículo.
# Universo: de -20 (muy rebasado) a 50 (distancia muy segura)
distancia_seguridad = ctrl.Antecedent(np.arange(-20, 51, 1), 'distancia_seguridad')



# 1. 'negativa': el ADS ha rebasado claramente al otro vehículo
# Pertenencia máxima desde -20 hasta -10, difusa hasta -5
distancia_seguridad['negativa'] = fuzz.trapmf(distancia_seguridad.universe, [-20, -20, -10, -5])

# 2. 'negativa_corta': el ADS ha rebasado recientemente o está casi paralelo al otro vehículo
# Transición entre negativo y cero (zona crítica)
distancia_seguridad['negativa_corta'] = fuzz.trimf(distancia_seguridad.universe, [-10, -5, 0])

# 3. 'corta': distancias frontales muy reducidas, riesgo de colisión
# Centro en 5 m, típica de conducción urbana o tráfico denso
distancia_seguridad['corta'] = fuzz.trimf(distancia_seguridad.universe, [0, 5, 10])

# 4. 'media': rango intermedio, posible pero depende de la velocidad
# Centro en 18 m
distancia_seguridad['media'] = fuzz.trimf(distancia_seguridad.universe, [8, 18, 28])

# 5. 'larga': zona de confort, distancia segura para acelerar o cambiar de carril
# Pertenencia máxima desde 40 m hasta 50 m
distancia_seguridad['larga'] = fuzz.trapmf(distancia_seguridad.universe, [30, 40, 50, 50])


distancia_seguridad.view()



## Decisión Acción:

In [ ]:
# Definición de la variable difusa de salida: "decision_accion"
# Esta variable representa la acción que tomará el vehículo autónomo en términos de aceleración o frenado.
# Está expresada en m/s², que es la unidad estándar para aceleración lineal.
# El rango va desde -3 (frenado fuerte) hasta +3 (aceleración fuerte), usando pasos de 0.1 para una buena resolución en el control.
decision_accion = ctrl.Consequent(np.arange(-3, 3.1, 0.1), 'decision_accion')

# Funciones de membresía para categorizar las posibles acciones del vehículo en tres niveles básicos.

# 'frenar': representa una decisión de desacelerar activamente, es decir, frenar.
# Esta función de membresía triangular cubre desde -3 hasta -1, con valor máximo (mayor pertenencia) en -2 m/s².
# Se activa cuando el sistema detecta riesgo por proximidad o velocidad alta del otro vehículo.
decision_accion['frenar'] = fuzz.trimf(decision_accion.universe, [-3, -2, -1])

# 'mantener': indica que el vehículo debe mantener su velocidad actual, sin frenar ni acelerar de forma significativa.
# Esta etiqueta cubre un rango centrado en 0 m/s², lo que representa una acción neutra.
# Ideal en situaciones de estabilidad, por ejemplo, cuando las velocidades y distancias son equilibradas.
decision_accion['mantener'] = fuzz.trimf(decision_accion.universe, [-1, 0, 1])

# 'acelerar': representa una decisión de aumentar la velocidad.
# La función triangular va de 1 a 3 m/s², con valor máximo en 2 m/s².
# Suele activarse cuando la distancia de seguridad es alta y/o la velocidad relativa es baja (el otro vehículo está lejos o va lento).
decision_accion['acelerar'] = fuzz.trimf(decision_accion.universe, [1, 2, 3])

# Graficamos
decision_accion.view()


# Definimos las reglas difusas

| Regla | Velocidad Relativa | Distancia Seguridad   | Acción         | Interpretación                                                                 |
|-------|---------------------|------------------------|----------------|---------------------------------------------------------------------------------|
| R1    | lenta            | Negativa               | Acelerar       | El otro va más rápido y ya lo has rebasado mucho → riesgo de ser alcanzado → acelerar|
| R2    | lenta            | Negativa_corta         | Acelerar         | El otro va más rápido y está muy cerca desde atrás (rebase reciente) → acelerar|
| R3    | lenta            | Corta                  | Frenar         | El otro va más rápido y estás cerca → frenar |
| R4    | lenta            | Media                  | Mantener         | El otro va más rápido y hay distancia media → mantener|
| R5    | lenta            | Larga                  | Mantener       | El otro va más rápido y hay mucha distancia → mantener|
| R6    | igual                | Negativa               | Mantener       | Lo has rebasado y vais igual → mantener                                         |
| R7    | igual                | Negativa_corta         | Acelerar       | Rebase reciente, misma velocidad → acelerar                                     |
| R8    | igual                | Corta                  | Acelerar       | Cerca pero misma velocidad → acelerar                               |
| R9    | igual                | Media                  | Mantener       | Igual velocidad y distancia media → mantener                                                               |
| R10   | igual                | Larga                  | Mantener       | Situación ideal → mantener                                            |
| R11   | rápida            | Negativa               | Mantener       | Tú vas más rápido y ya lo rebasaste bien → mantener |
| R12   | rápida            | Negativa_corta         | Acelerar       | Tú vas más rápido y estás justo rebasando → acelerar|
| R13   | rápida            | Corta                  | Frenar         | Lo estás alcanzando rápido estando muy cerca → frenar                          |
| R14   | rápida            | Media                  | Mantener       | Vas más rápido, pero con distancia aceptable → mantener                         |
| R15   | rápida            | Larga                  | Mantener       | Tú vas más rápido y hay mucha distancia → mantener |




In [ ]:
# 1. El otro va más rápido y ya lo has rebasado mucho → riesgo de ser alcanzado → acelerar
regla1 = ctrl.Rule(velocidad_relativa['lento'] & distancia_seguridad['negativa'], decision_accion['acelerar'])

# 2. El otro va más rápido y está muy cerca desde atrás (rebase reciente) → acelerar
regla2 = ctrl.Rule(velocidad_relativa['lento'] & distancia_seguridad['negativa_corta'], decision_accion['acelerar'])

# 3. El otro va más rápido y estás cerca → frenar
regla3 = ctrl.Rule(velocidad_relativa['lento'] & distancia_seguridad['corta'], decision_accion['frenar'])

# 4. El otro va más rápido y hay distancia media → mantener
regla4 = ctrl.Rule(velocidad_relativa['lento'] & distancia_seguridad['media'], decision_accion['mantener'])

# 5. El otro va más rápido y hay mucha distancia → mantener
regla5 = ctrl.Rule(velocidad_relativa['lento'] & distancia_seguridad['larga'], decision_accion['mantener'])


# 6. Ambos van igual de rápido y ya has rebasado → mantener
regla6 = ctrl.Rule(velocidad_relativa['igual'] & distancia_seguridad['negativa'], decision_accion['mantener'])

# 7. Igual velocidad y rebase reciente → acelerar
regla7 = ctrl.Rule(velocidad_relativa['igual'] & distancia_seguridad['negativa_corta'], decision_accion['acelerar'])

# 8. Igual velocidad y distancia corta → acelerar
regla8 = ctrl.Rule(velocidad_relativa['igual'] & distancia_seguridad['corta'], decision_accion['acelerar'])

# 9. Igual velocidad y distancia media → mantener
regla9 = ctrl.Rule(velocidad_relativa['igual'] & distancia_seguridad['media'], decision_accion['mantener'])

# 10. Igual velocidad y distancia larga → mantener
regla10 = ctrl.Rule(velocidad_relativa['igual'] & distancia_seguridad['larga'], decision_accion['mantener'])


# 11. Tú vas más rápido y ya lo rebasaste bien → mantener
regla11 = ctrl.Rule(velocidad_relativa['rapido'] & distancia_seguridad['negativa'], decision_accion['mantener'])

# 12. Tú vas más rápido y estás justo rebasando → acelerar
regla12 = ctrl.Rule(velocidad_relativa['rapido'] & distancia_seguridad['negativa_corta'], decision_accion['acelerar'])

# 13. Tú vas más rápido y estás cerca → frenar (riesgo de alcance)
regla13 = ctrl.Rule(velocidad_relativa['rapido'] & distancia_seguridad['corta'], decision_accion['frenar'])

# 14. Tú vas más rápido y hay distancia media → mantener
regla14 = ctrl.Rule(velocidad_relativa['rapido'] & distancia_seguridad['media'], decision_accion['mantener'])

# 15. Tú vas más rápido y hay mucha distancia → mantener
regla15 = ctrl.Rule(velocidad_relativa['rapido'] & distancia_seguridad['larga'], decision_accion['mantener'])


# Sistema de control difuso

In [ ]:


# Crear el sistema de control difuso con las 12 reglas
sistema_ctrl = ctrl.ControlSystem([
    regla1, regla2, regla3, regla4, regla5,
    regla6, regla7, regla8, regla9, regla10,
    regla11, regla12, regla13, regla14, regla15
])


def actualizar_aceleracion_B(a_B_anterior, tendencia=0.9, sigma=0.3, a_max=2.0, alpha=0.95):
    """
    Devuelve una aceleración suavizada para el vehículo tripulado.
    
    - tendencia: favorece mantener la dirección de aceleración anterior.
    - sigma: variación aleatoria (cuanto menor, más suave).
    - alpha: peso de la aceleración previa (cuanto mayor, más suavidad).
    """
    # Generar nueva aceleración con sesgo hacia la tendencia anterior
    media = tendencia * a_B_anterior
    a_B_generada = np.random.normal(loc=media, scale=sigma)


    # Aplicar suavizado exponencial
    a_B_suave = alpha * a_B_anterior + (1 - alpha) * a_B_generada

    # Restringir a rango físico razonable
    return np.clip(a_B_suave, -a_max, a_max)



def calcular_trayectoria_difusa_doble(x_A, v_A, a_A, x_B, v_B, a_B, t_ini, t_max, inf_t=0.01, v_min=12.5, v_max=25):
    simulador = ctrl.ControlSystemSimulation(sistema_ctrl)
    resultados = [(x_A, v_A, a_A, x_B, v_B, a_B, t_ini)]

    while t_ini < t_max:
        # Calcular distancia y velocidad relativa
        distancia = x_B - x_A
        velocidad_rel = v_B - v_A

        # Entradas al sistema difuso
        simulador.input['velocidad_relativa'] = np.clip(velocidad_rel, -30, 30)
        simulador.input['distancia_seguridad'] = np.clip(distancia, -10, 50)

        try:
            simulador.compute()
            a_A = simulador.output['decision_accion']
        except KeyError:
            a_A = 0

        # Actualizar el estado del vehículo autónomo
        v_A = np.clip(v_A + a_A * inf_t, v_min, v_max)
        x_A += v_A * inf_t

        # Actualizar aceleración y estado del vehículo B (tripulado)
        a_B = actualizar_aceleracion_B(a_B)
        v_B += a_B * inf_t
        x_B += v_B * inf_t

        # Avanzar tiempo
        t_ini += inf_t
        #decision_accion.view(sim=sistema_ctrl)
        # Guardar estado actual
        resultados.append((x_A, v_A, a_A, x_B, v_B, a_B, t_ini))

    return resultados




In [ ]:
def calcular_trayectoria_difusa_gif(x_A, v_A, a_A, x_B, v_B, a_B, t_ini, t_max, inf_t=0.01,
                                     v_min=12.5, v_max=25, gif_name="sistema_difuso.gif", fps=10):
    simulador = ctrl.ControlSystemSimulation(sistema_ctrl)
    resultados = [(x_A, v_A, a_A, x_B, v_B, a_B, t_ini)]
    frames = []

    while t_ini < t_max:
        distancia = x_B - x_A
        velocidad_rel = v_B - v_A

        simulador.input['velocidad_relativa'] = np.clip(velocidad_rel, -20, 20)
        simulador.input['distancia_seguridad'] = np.clip(distancia, -10, 50)

        try:
            simulador.compute()
            a_A = simulador.output['decision_accion']
        except KeyError:
            a_A = 0

        # Capturar visualización del sistema difuso
        fig = decision_accion.view(sim=simulador)
        fig = plt.gcf()
        buf = io.BytesIO()
        fig.savefig(buf, format='png')
        buf.seek(0)
        image = imageio.v2.imread(buf)
        frames.append(image)
        plt.close(fig)

        # Actualización de vehículos
        v_A = np.clip(v_A + a_A * inf_t, v_min, v_max)
        x_A += v_A * inf_t

        a_B = actualizar_aceleracion_B(a_B)
        v_B += a_B * inf_t
        x_B += v_B * inf_t

        t_ini += inf_t
        resultados.append((x_A, v_A, a_A, x_B, v_B, a_B, t_ini))

    # Crear el GIF
    imageio.mimsave(gif_name, frames, fps=fps)

    return resultados


### Ejemplos 1 y 2:

In [ ]:
# Parámetros iniciales
x_A_inicial = 15
v_A_inicial = 20
x_B_inicial = 0
v_B_inicial = 24
a_A_inicial = 0
a_B_inicial = 0
t_max = 10

trayectoria_difusa = calcular_trayectoria_difusa_doble(
    x_A_inicial, v_A_inicial, a_A_inicial,
    x_B_inicial, v_B_inicial, a_B_inicial,
    0, t_max
)


# Extraer datos
tiempos = [t for _, _, _, _, _, _, t in trayectoria_difusa]
pos_autonomo = [x for x, _, _, _, _, _, _ in trayectoria_difusa]
pos_convencional = [x for _, _, _, x, _, _, _ in trayectoria_difusa]
vel_autonomo = [v for _, v, _, _, _, _, _ in trayectoria_difusa]
vel_convencional = [v for _, _, _, _, v, _, _ in trayectoria_difusa]
aceleracion_autonomo = [a for _, _, a, _, _, _, _ in trayectoria_difusa]
aceleracion_tripulado = [a for _, _, _, _, _, a, _ in trayectoria_difusa]

# Añadir una tercera gráfica para la aceleración
plt.figure(figsize=(18, 5))

# Posición
plt.subplot(1, 3, 1)
plt.plot(tiempos, pos_autonomo, label='Autónomo (x_A)', linewidth=2, color='blue')
plt.plot(tiempos, pos_convencional, label='Convencional (x_B)', linestyle='--', color='red')
plt.xlabel('Tiempo (s)')
plt.ylabel('Posición (m)')
plt.title('Evolución de las posiciones')
plt.legend()
plt.grid()

# Velocidad
plt.subplot(1, 3, 2)
plt.plot(tiempos, vel_autonomo, label='Velocidad Autónomo (v_A)', color='blue')
plt.plot(tiempos, vel_convencional, label='Velocidad Convencional (v_B)', linestyle='--', color='red')
plt.xlabel('Tiempo (s)')
plt.ylabel('Velocidad (m/s)')
plt.ylim(-20, 40)
plt.title('Evolución de las velocidades')
plt.legend()
plt.grid()

# Aceleración
plt.subplot(1, 3, 3)
plt.plot(tiempos, aceleracion_autonomo, label='Aceleración Autónomo (a_A)', color='blue')
plt.plot(tiempos, aceleracion_tripulado, label='Aceleración Convencional (a_B)', linestyle='--', color='red')
plt.xlabel('Tiempo (s)')
plt.ylabel('Aceleración (m/s²)')
plt.ylim(-3, 3)
plt.title('Evolución de las aceleraciones')
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()



#### Podemos crear un gift con la función de pertencia de la decisión

In [ ]:
#trayectoria_difusa_plot = calcular_trayectoria_difusa_gif(
#    x_A_inicial, v_A_inicial, a_A_inicial,
#    x_B_inicial, v_B_inicial, a_B_inicial,
#    0, t_max
#)


## Comprobar la factibilidad de la maniobra

In [ ]:
## Definamos una función que cuente los True's consecutivos en una lista de pyhton

def contar_consecutivos(lista, num_consecutivos):
    contador_consecutivos = 0
    secuencias = []
    indices_inicio = []

    for i, valor in enumerate(lista):
        if valor:
            if contador_consecutivos == 0:
                inicio = i  # Marca la posición de inicio de una nueva secuencia de consecutivos
            contador_consecutivos += 1
        else:
            if contador_consecutivos > num_consecutivos:
                secuencias.append(contador_consecutivos)
                indices_inicio.append(inicio)
            contador_consecutivos = 0

    # Verificar al final de la lista si los consecutivos son mayores al número que buscamos
    if contador_consecutivos > num_consecutivos:
        secuencias.append(contador_consecutivos)
        indices_inicio.append(inicio)

    # Si no hay, la lista secuencia y la lista indices_inicio serán listas vacías

    return secuencias, indices_inicio

In [ ]:
def evaluar_maniobra(trayectoria, d_seguridad=5, t_maniobra_min=4, inf_t=0.01, x_final=np.inf):
    """
    Evalúa si en la trayectoria simulada se cumple la distancia de seguridad durante al menos t_maniobra_min segundos,
    y antes de que el vehículo autónomo supere el final del carril de aceleración.

    Parámetros:
    - trayectoria: lista de tuplas (x_A, v_A, a_A, x_B, v_B, a_B, t)
    - d_seguridad: distancia mínima que debe mantenerse entre vehículos (m)
    - t_maniobra_min: tiempo mínimo necesario para considerar una maniobra segura (s)
    - inf_t: paso temporal de simulación
    - x_final: posición donde termina el carril de aceleración (m)
    """

    lista_valores_np = np.array(trayectoria)

    lista_valores_np = lista_valores_np[lista_valores_np[:,0] <= x_final]

    # En primer lugar calculamos el valor absoluto de las diferencias
    diferencia = abs(lista_valores_np[:,0] - lista_valores_np[:,3])
    diferencia = diferencia.reshape(-1, 1)

    # Añadimos la columna a nuestra matriz
    lista_valores_np = np.hstack((lista_valores_np, diferencia))

    # Ahora veamos qué distancias son superiores a la distancia de seguridad (5 (metros))
    superiores = diferencia > d_seguridad
    superiores.reshape(-1,1)
    lista_valores_np = np.hstack((lista_valores_np, superiores))

    ## Ahora tenemos que ver que esa condición se cumpla durante el t_maniobra_min
    # Para ello tenemos que tener en cuenta de cuánto es nuestro infinitesimal de orden t para
    # calcular el número de True's que tienen que ir seguidos en la columna superiores

    # Calculemos cuantos infinitesimales con True tienen que ir seguidos en funcion de nuestro t_maniobra_min
    num_seguidos = t_maniobra_min/inf_t

    ## Usemos la función creada anteriormente para contar los valores consecutivos de nuestra lista superiores
    secuencias, indices_inicio_secuencia = contar_consecutivos(superiores, num_seguidos)

    if secuencias != []:
        # Obtengamos las posiciones y los tiempos en los que el vehículo autónomo puede comenzar la maniobra, así como los
        # segundos de los que dispondrá para realizar dicha maniobra
        resultados = [['Maniobra', "x_a", "v_a", "a_a", "x_h", "v_h", "a_h", "t_inicio_maniobra", "dist_coches", "bool", "Tiempo para la maniobra", "Velocidad ADS final maniobra"]]

        for indice in range(len(secuencias)):
            tiempo_disp_maniobra = secuencias[indice]*inf_t
            i_ini_maniobra = indices_inicio_secuencia[indice]
            velocidad_a_final_maniobra = lista_valores_np[i_ini_maniobra + int(num_seguidos)].tolist()[1] # Sumamos num_seguidos para saber la velocidad final de la maniobra
            valores_vehiculos = lista_valores_np[i_ini_maniobra].tolist()
            resultado_maniobra = [True] + valores_vehiculos + [tiempo_disp_maniobra] + [velocidad_a_final_maniobra]
            resultados.append(resultado_maniobra)

        #print("Se puede realizar la maniobra")
        return pd.DataFrame(np.array(resultados[1:]), columns = np.array(resultados[0]))
        ### return("Se puede realizar la maniobra", pd.DataFrame(np.array(resultados[1:]), columns = np.array(resultados[0])))
        #return(pd.DataFrame(lista_valores_np))

    else:
        resultados = [['Maniobra']]
        ### return("No se puede realizar la maniobra")
        resultados.append([False])
        return(pd.DataFrame(np.array(resultados[1:]), columns = np.array(resultados[0])))

In [ ]:
evaluar_maniobra(trayectoria_difusa, x_final = 200)

In [ ]:
evaluar_maniobra(trayectoria_difusa, x_final=120)

### Ejemplos 3 y 4



In [ ]:
# Parámetros iniciales
x_A_inicial = 0
v_A_inicial = 20
x_B_inicial = 5
v_B_inicial = 24
a_A_inicial = 0
a_B_inicial = 3
t_max = 10

trayectoria_difusa_2 = calcular_trayectoria_difusa_doble(
    x_A_inicial, v_A_inicial, a_A_inicial,
    x_B_inicial, v_B_inicial, a_B_inicial,
    0, t_max
)

# Extraer datos
tiempos = [t for _, _, _, _, _, _, t in trayectoria_difusa_2]
pos_autonomo = [x for x, _, _, _, _, _, _ in trayectoria_difusa_2]
pos_convencional = [x for _, _, _, x, _, _, _ in trayectoria_difusa_2]
vel_autonomo = [v for _, v, _, _, _, _, _ in trayectoria_difusa_2]
vel_convencional = [v for _, _, _, _, v, _, _ in trayectoria_difusa_2]
aceleracion_autonomo = [a for _, _, a, _, _, _, _ in trayectoria_difusa_2]
aceleracion_tripulado = [a for _, _, _, _, _, a, _ in trayectoria_difusa_2]

# Añadir una tercera gráfica para la aceleración
plt.figure(figsize=(18, 5))

# Posición
plt.subplot(1, 3, 1)
plt.plot(tiempos, pos_autonomo, label='Autónomo (x_A)', linewidth=2, color='blue')
plt.plot(tiempos, pos_convencional, label='Convencional (x_B)', linestyle='--', color='red')
plt.xlabel('Tiempo (s)')
plt.ylabel('Posición (m)')
plt.title('Evolución de la Posición')
plt.legend()
plt.grid()

# Velocidad
plt.subplot(1, 3, 2)
plt.plot(tiempos, vel_autonomo, label='Velocidad Autónomo (v_A)', color='blue')
plt.plot(tiempos, vel_convencional, label='Velocidad Convencional (v_B)', linestyle='--', color='red')
plt.xlabel('Tiempo (s)')
plt.ylabel('Velocidad (m/s)')
plt.ylim(-20, 40)
plt.title('Evolución de la Velocidad del ADS')
plt.legend()
plt.grid()

# Aceleración
plt.subplot(1, 3, 3)
plt.plot(tiempos, aceleracion_autonomo, label='Aceleración Autónomo (a_A)', color='blue')
plt.plot(tiempos, aceleracion_tripulado, label='Aceleración Convencional (a_B)', linestyle='--', color='red')
plt.xlabel('Tiempo (s)')
plt.ylabel('Aceleración (m/s²)')
plt.ylim(-3, 3)
plt.title('Evolución de la Aceleración del ADS')
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()


In [ ]:
evaluar_maniobra(trayectoria_difusa_2, x_final = 200)

In [ ]:
evaluar_maniobra(trayectoria_difusa_2, x_final = 120)

In [ ]:
trayectoria_difusa_plot = calcular_trayectoria_difusa_gif(
    x_A_inicial, v_A_inicial, a_A_inicial,
    x_B_inicial, v_B_inicial, a_B_inicial,
    0, t_max
)


In [ ]:
# Parámetros iniciales
x_A_inicial = 0
v_A_inicial = 20
x_B_inicial = 5
v_B_inicial = 24
a_A_inicial = 0
a_B_inicial = -3
t_max = 10

trayectoria_difusa_2 = calcular_trayectoria_difusa_doble(
    x_A_inicial, v_A_inicial, a_A_inicial,
    x_B_inicial, v_B_inicial, a_B_inicial,
    0, t_max
)

# Extraer datos
tiempos = [t for _, _, _, _, _, _, t in trayectoria_difusa_2]
pos_autonomo = [x for x, _, _, _, _, _, _ in trayectoria_difusa_2]
pos_convencional = [x for _, _, _, x, _, _, _ in trayectoria_difusa_2]
vel_autonomo = [v for _, v, _, _, _, _, _ in trayectoria_difusa_2]
vel_convencional = [v for _, _, _, _, v, _, _ in trayectoria_difusa_2]
aceleracion_autonomo = [a for _, _, a, _, _, _, _ in trayectoria_difusa_2]
aceleracion_tripulado = [a for _, _, _, _, _, a, _ in trayectoria_difusa_2]

# Añadir una tercera gráfica para la aceleración
plt.figure(figsize=(18, 5))

# Posición
plt.subplot(1, 3, 1)
plt.plot(tiempos, pos_autonomo, label='Autónomo (x_A)', linewidth=2, color='blue')
plt.plot(tiempos, pos_convencional, label='Convencional (x_B)', linestyle='--', color='red')
plt.xlabel('Tiempo (s)')
plt.ylabel('Posición (m)')
plt.title('Evolución de la Posición')
plt.legend()
plt.grid()

# Velocidad
plt.subplot(1, 3, 2)
plt.plot(tiempos, vel_autonomo, label='Velocidad Autónomo (v_A)', color='blue')
plt.plot(tiempos, vel_convencional, label='Velocidad Convencional (v_B)', linestyle='--', color='red')
plt.xlabel('Tiempo (s)')
plt.ylabel('Velocidad (m/s)')
plt.ylim(-20, 40)
plt.title('Evolución de la Velocidad del ADS')
plt.legend()
plt.grid()

# Aceleración
plt.subplot(1, 3, 3)
plt.plot(tiempos, aceleracion_autonomo, label='Aceleración Autónomo (a_A)', color='blue')
plt.plot(tiempos, aceleracion_tripulado, label='Aceleración Convencional (a_B)', linestyle='--', color='red')
plt.xlabel('Tiempo (s)')
plt.ylabel('Aceleración (m/s²)')
plt.ylim(-3, 3)
plt.title('Evolución de la Aceleración del ADS')
plt.legend()
plt.grid()

plt.tight_layout()
plt.show()


In [ ]:
trayectoria_difusa_plot = calcular_trayectoria_difusa_gif(
    x_A_inicial, v_A_inicial, a_A_inicial,
    x_B_inicial, v_B_inicial, a_B_inicial,
    0, t_max
)

In [ ]:
evaluar_maniobra(trayectoria_difusa_2, x_final = 200)

In [ ]:
evaluar_maniobra(trayectoria_difusa_2, x_final = 120)

## Hagamos un gift con las trayectorias

In [ ]:
## Creamos una función que recorra una lista que nos añada a la posición y lo que necesitamos


# Maniobra cambio de carril: 4 segundos
# Pasar en ese tiempo de y=0 a y=1

def obtener_coordenadas_y(len_x, t_0):
    '''
    Esta función devuelve la lista de valores del eje y para realizar la maniobra.

    len_x es la longitud de la lista de valores del eje x.
    t_0 es el instante de tiempo (en segundos) en el que se inicia la maniobra.
    '''

    # Crear la lista de valores inicializada en 0
    y_ADS = list(np.zeros(len_x))

    # i es el índice de inicio en el tiempo t_0
    i = int(t_0 * 100)

    # Definir el incremento de la posición por cada paso
    incremento = 0.002

    # Asegurarse de que la maniobra no se salga de los límites de la lista
    max_pasos = min(400, len_x - i)  # 400 pasos corresponden a 4 segundos

    # Realizar la maniobra incrementando el valor
    for paso in range(max_pasos):
        y_ADS[i + paso] = incremento * paso

    # Si llegamos al final de la maniobra, completar con 1 el resto
    for j in range(i + max_pasos, len_x):
        y_ADS[j] = 1

    return y_ADS


In [ ]:
# Posición final del carril de aceleración
pos_final_carril = 200


# Posiciones de los dos vehículos
posicion_x_ADS = np.array(trayectoria_difusa)[:,0]
posicion_x_VC = np.array(trayectoria_difusa)[:,3]

# Tiempo de inicio de la maniobra
t_inicio_maniobra = evaluar_maniobra(trayectoria_difusa, x_final = pos_final_carril)["t_inicio_maniobra"].values[0]

# Posiciones del coche A (ADS)
x_A = list(posicion_x_ADS)
y_A = obtener_coordenadas_y(len(x_A),t_inicio_maniobra)

# Posiciones del coche B (VC)
x_B = list(posicion_x_VC)
y_B = list(np.ones(len(posicion_x_ADS)))

In [ ]:
# Crear una lista para guardar las imágenes
imagenes = []

# Crear gráficos para cada paso en el tiempo
for i in range(len(x_A)):
    plt.figure(figsize=(12, 6))
    plt.xlim(0, 226)  # Ajusta los límites según tus datos reales
    plt.ylim(-1, 2)
    plt.title("Ejemplo 1: Carril de aceleración de 200 metros")

    # Establecer el fondo gris
    plt.gca().set_facecolor('gray')

    # Ocultar las etiquetas y ticks del eje y
    plt.yticks([])  # Esto elimina las etiquetas y los ticks

    # Agregar líneas de carretera
    plt.plot([0, 180], [-0.5, -0.5], color='white', linestyle='-', linewidth=1)
    plt.plot([199.5, 250],[0.5, 0.5], color='white', linestyle='-', linewidth=1)
    plt.plot([180, 200.5],[-0.5, 0.5], color='white', linestyle='-', linewidth=1)
    plt.plot([0, 200.5],[0.5, 0.5], color='white', linestyle='--', linewidth=1)
    plt.axhline(y=1.5, color='white', linestyle='-', linewidth=1)

    # Graficar posiciones de los coches en el paso actual
    plt.scatter(x_A[i], y_A[i], c='blue', label='ADS', s=500)
    plt.scatter(x_B[i], y_B[i], c='red', label='VC', s=500)

    # Añadir leyenda
    plt.legend(prop={'size': 19})

    # Guardar la figura en un archivo temporal y agregarla a la lista de imágenes
    filename = f'temp_{i}.png'
    plt.savefig(filename)
    imagenes.append(imageio.imread(filename))

    # Cerrar la figura para no sobrecargar memoria
    plt.close()

# Crear un GIF a partir de las imágenes
imageio.mimsave('animacion_coches.gif', imagenes, duration=0.00001)


### Carril 120 metros

In [ ]:
# Posición final del carril de aceleración
pos_final_carril = 120


# Posiciones de los dos vehículos
posicion_x_ADS = np.array(trayectoria_difusa)[:,0]
posicion_x_VC = np.array(trayectoria_difusa)[:,3]

# Tiempo de inicio de la maniobra
t_inicio_maniobra = evaluar_maniobra(trayectoria_difusa, x_final = pos_final_carril)["t_inicio_maniobra"].values[0]

# Posiciones del coche A (ADS)
x_A = list(posicion_x_ADS)
y_A = obtener_coordenadas_y(len(x_A),t_inicio_maniobra)

# Posiciones del coche B (VC)
x_B = list(posicion_x_VC)
y_B = list(np.ones(len(posicion_x_ADS)))

In [ ]:
# Crear una lista para guardar las imágenes
imagenes = []

# Crear gráficos para cada paso en el tiempo
for i in range(len(x_A)):
    plt.figure(figsize=(12, 6))
    plt.xlim(0, 226)  # Ajusta los límites según tus datos reales
    plt.ylim(-1, 2)
    plt.title("Ejemplo 1: Carril de aceleración de 120 metros")

    # Establecer el fondo gris
    plt.gca().set_facecolor('gray')

    # Ocultar las etiquetas y ticks del eje y
    plt.yticks([])  # Esto elimina las etiquetas y los ticks

    # Agregar líneas adicionales
    plt.plot([0, 100], [-0.5, -0.5], color='white', linestyle='-', linewidth=1)
    plt.plot([120, 250],[0.5, 0.5], color='white', linestyle='-', linewidth=1)
    plt.plot([100, 120],[-0.5, 0.5], color='white', linestyle='-', linewidth=1)
    plt.plot([0, 120],[0.5, 0.5], color='white', linestyle='--', linewidth=1)
    plt.axhline(y=1.5, color='white', linestyle='-', linewidth=1)

    # Graficar posiciones de los coches en el paso actual
    plt.scatter(x_A[i], y_A[i], c='blue', label='ADS', s=500)
    plt.scatter(x_B[i], y_B[i], c='red', label='VC', s=500)

    # Añadir leyenda
    plt.legend(prop={'size': 19})

    # Guardar la figura en un archivo temporal y agregarla a la lista de imágenes
    filename = f'temp_{i}.png'
    plt.savefig(filename)
    imagenes.append(imageio.imread(filename))

    # Cerrar la figura para no sobrecargar memoria
    plt.close()

# Crear un GIF a partir de las imágenes
imageio.mimsave('animacion_coches.gif', imagenes, duration=0.00001)
